<a href="https://colab.research.google.com/github/nagusubra/Solar_panel_dust_detection/blob/main/Solar_panel_dust_detection_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization
!pip install openpyxl
!pip install xlsxwriter

import xlsxwriter
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 3.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
import tensorflow_datasets as tfds
import zipfile

In [3]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation function

In [4]:
# Evaluate Model Size
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)


def evaluate_model(model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df


def evaluate_model_without_saving_stats(model, model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  # model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df




# research paper evaluation function
def evaluate_research_paper_with_model_path(model_path):
  # model_path = "/content/models/solnet.hdf5"
  solnet = load_model(model_path, compile=False)
  history = solnet.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

def evaluate_research_paper_with_model(model):
  history = model.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

In [9]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

# Load data

In [ ]:
batch_size = 32
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)

# Load base model and Evaluate


In [5]:
# final_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/solnetOriginal_final_base_model.hdf5'
final_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/final_base_model.h5'
final_model = tf.keras.models.load_model(final_model_path)

In [7]:
test_acc, model_size, executionTime, evulation_df = evaluate_model_without_saving_stats(final_model, final_model_path, "#0", val_dataset)
print(test_acc)

9/9 [==============================] - 29s 37ms/step

Model Accuracy: 96.18055820465088 %
Model Size: 648760230.00 bytes
Inference Time is:  3.2716289361317954 s
0.9618055820465088


In [8]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,96.18055820465088 %
3,Loss,11.30877360701561 %
4,Model Size,648760230 bytes
5,Inference Time,3.2716289361317954 sec


# Load iteratively pruned model and Evaluate

In [16]:
iteratively_pruned_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/pruned_model_saprsity_50.h5'
iteratively_pruned_model = tf.keras.models.load_model(iteratively_pruned_model_path)

iteratively_pruned_stripped_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/stripped_pruned_model_saprsity_70.h5'
# iteratively_pruned_stripped_model = tf.keras.models.load_model(iteratively_pruned_stripped_model_path)

ValueError: ignored

In [13]:
pruned_test_acc, pruned_model_size, pruned_executionTime, pruned_evulation_df = evaluate_model_without_saving_stats(iteratively_pruned_stripped_model, iteratively_pruned_stripped_model_path, "#0", val_dataset)
print(pruned_test_acc)

RuntimeError: ignored

In [ ]:
pruned_evulation_df

In [ ]:
print_model_weights_sparsity(iteratively_pruned_model)

conv2d_10/kernel:0: 89.01% sparsity  (31019/34848)
conv2d_11/kernel:0: 89.01% sparsity  (546892/614400)
conv2d_12/kernel:0: 89.01% sparsity  (787524/884736)
conv2d_13/kernel:0: 89.01% sparsity  (1181286/1327104)
conv2d_14/kernel:0: 89.01% sparsity  (787524/884736)
dense_6/kernel:0: 89.01% sparsity  (33601033/37748736)
dense_7/kernel:0: 89.01% sparsity  (14933793/16777216)
dense_8/kernel:0: 89.01% sparsity  (3646/4096)


In [ ]:
test_loss_pruned_50, test_acc_pruned_50 = pruned_model_saprsity_50.evaluate(val_dataset, verbose=0)

In [ ]:
startTime = time.time()
prediction = pruned_model_saprsity_50.predict(val_dataset)
executionTimePruned50 = (time.time() - startTime)/len(val_dataset)

9/9 [==============================] - 24s 36ms/step


In [ ]:
pruned_model_size = get_gzipped_model_size('stripped_pruned_model_saprsity_50.h5')

In [ ]:
print('\nPruned Model Accuracy:', test_acc_pruned_50*100, '%')
print("Pruned Model Size: %.2f bytes" % (pruned_model_size))
print("Pruned Inference Time is", executionTimePruned50, "s")


Pruned Model Accuracy: 50.0 %
Pruned Model Size: 46415300.00 bytes
Pruned Inference Time is 2.6262646516164145 s


In [ ]:
pruned_test_acc, pruned_model_size, pruned_executionTime, pruned_evulation_df = evaluate_model_without_saving_stats(pruned_model_saprsity_50, 'stripped_pruned_model_saprsity_50.h5', "#1", val_dataset)
print(pruned_test_acc)

9/9 [==============================] - 23s 35ms/step

Model Accuracy: 50.0 %
Model Size: 46415300.00 bytes
Inference Time is:  2.5482777489556208 s
0.5


In [ ]:
pruned_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#1
2,Accuracy,50.0 %
3,Loss,282.0566654205322 %
4,Model Size,46415300 bytes
5,Inference Time,2.5482777489556208 sec


In [ ]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,96.18055820465088 %
3,Loss,11.30877360701561 %
4,Model Size,648760230 bytes
5,Inference Time,2.708994812435574 sec


In [ ]:
pruned_model_saprsity_50.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/pruned_model_saprsity_90.h5')
stripped_pruned_model_saprsity_50.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/stripped_pruned_model_saprsity_90.h5')

In [ ]:
pruned_model_saprsity_50.save('pruned_model_saprsity_90.h5')
stripped_pruned_model_saprsity_50.save('stripped_pruned_model_saprsity_90.h5')

## Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_to_quantize)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
fp16_quantized_tflite_model = converter.convert()

with open('fp16_quantized_tflite_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_model)
interpreter_fp16_quant = tf.lite.Interpreter(model_path=str('fp16_quantized_tflite_model.tflite'))
interpreter_fp16_quant.allocate_tensors()

evaluate_model(interpreter_fp16_quant, 'fp16_quantized_tflite_model.tflite')